In [23]:
import pandas as pd

df = pd.read_csv('/content/dataset.csv')
# Combine date and time into a single timestamp column
df['Timestamp'] = pd.to_datetime(
    df['LastUpdatedDate'] + ' ' + df['LastUpdatedTime'],
    format='%d-%m-%Y %H:%M:%S'
)
# Optional: drop the original date/time columns
df = df.drop(columns=['LastUpdatedDate', 'LastUpdatedTime'])

# Assuming df['Timestamp'] is already a pandas datetime column
# Convert to ISO 8601 with 'T' and 'Z' (UTC)
df['Timestamp'] = pd.to_datetime(df['Timestamp']).dt.strftime('%Y-%m-%dT%H:%M:%SZ')

df.to_csv('dataset_processed.csv', index=False)
# Assuming df['Timestamp'] is a pandas datetime column
df['Timestamp'] = pd.to_datetime(df['Timestamp'], utc=True)
df['Timestamp'] = df['Timestamp'].dt.strftime('%Y-%m-%dT%H:%M:%S+00:00')

df.to_csv('dataset_processed.csv', index=False)


In [24]:
traffic_map = {'low': 1, 'average': 2, 'high': 3}
df['TrafficLevel'] = df['TrafficConditionNearby'].map(traffic_map)


In [25]:
!pip install pathway
import pathway as pw

class ParkingSchema(pw.Schema):
    Timestamp: pw.schema.dt.DATE_TIME_UTC
    SystemCodeNumber: str
    Capacity: int
    Latitude: float
    Longitude: float
    Occupancy: int
    VehicleType: str
    TrafficLevel: int
    QueueLength: int
    IsSpecialDay: int

# Save processed CSV for Pathway
df.to_csv('dataset_processed.csv', index=False)



In [26]:

source = pw.io.csv.read(
    'dataset_processed.csv',
    schema=ParkingSchema,
    mode='static',  # or 'streaming' if simulating real-time
    autocommit_duration_ms=1000
)

In [27]:
BASE_PRICE = 10.0
ALPHA, BETA, GAMMA, DELTA, LAMBDA = 2.0, 0.8, 0.5, 1.0, 1.2
veh_weight = {'car': 1.0, 'bike': 0.5, 'truck': 1.5}

def model1(prev_price, occupancy, capacity):
    return prev_price + ALPHA * (occupancy / capacity)

def model2(base, occ, cap, queue, traffic, special, vtype):
    demand = (ALPHA * (occ / cap)
              + BETA * queue
              - GAMMA * traffic
              + DELTA * special
              + veh_weight.get(vtype, 1.0))
    # Normalize demand
    max_raw = ALPHA + BETA * 10 + veh_weight['truck']  # adjust max queue if needed
    norm = demand / max_raw
    factor = np.clip(1 + LAMBDA * norm, 0.5, 2.0)
    return base * factor


In [28]:
def pricing_logic(SystemCodeNumber, Capacity, Occupancy, QueueLength, TrafficLevel, IsSpecialDay, VehicleType):
    # For stateless logic (model2)
    return model2(BASE_PRICE, Occupancy, Capacity, QueueLength, TrafficLevel, IsSpecialDay, VehicleType)

priced = source.select(
    Timestamp=pw.this.Timestamp,
    SystemCodeNumber=pw.this.SystemCodeNumber,
    Price=pw.apply(
        pricing_logic,
        pw.this.SystemCodeNumber,
        pw.this.Capacity,
        pw.this.Occupancy,
        pw.this.QueueLength,
        pw.this.TrafficLevel,
        pw.this.IsSpecialDay,
        pw.this.VehicleType,
    ),
)

pw.io.csv.write(priced, 'output_priced.csv')



In [30]:
import numpy as np

pw.run()

Output()

In [31]:
import pandas as pd
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import ColumnDataSource
output_notebook()

df = pd.read_csv('output_priced.csv', parse_dates=['Timestamp'])
p = figure(x_axis_type='datetime', width=800, height=400, title='Dynamic Parking Prices')

for lid in df.SystemCodeNumber.unique():
    df_l = df[df.SystemCodeNumber == lid]
    p.line('Timestamp', 'Price', source=ColumnDataSource(df_l), legend_label=f'Lot {lid}')

p.legend.click_policy = 'hide'
show(p)
